In [1]:
import tensorflow as tf
import syft as sy
from syft.core.frameworks.torch import utils

/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
hook = sy.TorchHook()

In [3]:
me = hook.local_worker

In [4]:
me.is_client_worker = False

In [16]:
bob

<syft.core.workers.virtual.VirtualWorker id:bob>

In [5]:
bob = sy.VirtualWorker(id='bob')
bob.add_worker(me)
me.add_worker(bob)

In [6]:
def get_tf_override_function(attr):
    def method(*args, **kwargs):

        worker = None
        if('worker' in kwargs):
            worker = kwargs['worker']
            del kwargs['worker']        
        
        kwargs['name'] = worker.id + "_" + kwargs['name'] + "_" + str(random.randint(0,10e10))
        
        cmd, locations, owners = utils.compile_command(attr=str(attr),
                                                       args=args,
                                                       kwargs=kwargs,
                                                       has_self=False,
                                                       self=None)
        if(len(locations) > 0):
            worker = locations[0]
        elif worker is not None:
            ""
        else:
            worker = sy.local_worker

        result = sy.local_worker.send_command(recipient=worker,
                                              message=cmd,
                                              framework="tf")
        return result
        
    return method

In [7]:
import random

In [8]:
# tf.Tensor.__init__ = tf.Tensor.native__init__
# x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")

In [9]:
tf.Tensor.native__init__ = tf.Tensor.__init__

In [14]:


def new___init__(self, *args, **kwargs):
    print("initializing")

    if 'owner' in kwargs and kwargs['owner'] is not None:
        owner = kwargs['owner']
    else:
        owner = hook.local_worker
    
    self.native__init__(*args, *kwargs)
    
    owner.register_object(self)    
    
tf.Tensor.__init__ = new___init__

@property
def id(self):
    return self.name

tf.Tensor.id = id

@property
def owner(self):
    return sy.local_worker.get_worker(self.name.split("_")[0])

tf.Tensor.owner = owner

def create_pointer(self):
    return self

tf.Tensor.create_pointer = create_pointer

In [15]:
result = get_tf_override_function('placeholder')(tf.float32, shape=[4,2], name="x-input", worker=bob)

processing tf call
{'command': 'placeholder', 'has_self': False, 'args': ('tf.float32',), 'kwargs': {'shape': [4, 2], 'name': 'bob_x-input_41624060027'}}
process tf.function
initializing


In [12]:
bob._objects

{'bob_x-input_87279742196:0': <tf.Tensor 'bob_x-input_87279742196:0' shape=(4, 2) dtype=float32>}

In [13]:
me._objects

{}

In [18]:
me._objects['bob_x-input:0'].id

'bob_x-input:0'

In [46]:
class PointerVariable(tf.Variable):
    def __add__(self, other):
        return self.__sub__(other)
#     def __new__(self, *args, **kwargs):
#         super().__init__(self, *args, **kwargs)
#         print ("initializing")

In [47]:
pv = PointerVariable(tf.zeros([2]), name="pointer")

In [28]:
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
y_ = tf.placeholder(tf.float32, shape=[4,1], name="y-input")

initializing


AttributeError: 'Tensor' object has no attribute '_op'

In [11]:
x_

<tf.Tensor 'x-input_1:0' shape=(4, 2) dtype=float32>

In [25]:
Theta1 = tf.Variable(tf.random_uniform([2,2], -1, 1), name="Theta1")
Theta2 = tf.Variable(tf.random_uniform([2,1], -1, 1), name="Theta2")

In [26]:
Bias1 = tf.Variable(tf.zeros([2]), name="Bias1")
Bias2 = tf.Variable(tf.zeros([1]), name="Bias2")

In [27]:
A2 = tf.sigmoid(tf.matmul(x_, Theta1) + Bias1)
Hypothesis = tf.sigmoid(tf.matmul(A2, Theta2) + Bias2)

In [28]:
cost = tf.reduce_mean(( (y_ * tf.log(Hypothesis)) + 
        ((1 - y_) * tf.log(1.0 - Hypothesis)) ) * -1)

In [29]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [30]:
XOR_X = [[0,0],[0,1],[1,0],[1,1]]
XOR_Y = [[0],[1],[1],[0]]

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [31]:
for i in range(100000):
    sess.run(train_step, feed_dict={x_: XOR_X, y_: XOR_Y})
    
    if i % 1000 == 0:
        print('Epoch ', i)
#         print('Hypothesis ', sess.run(Hypothesis, feed_dict={x_: XOR_X, y_: XOR_Y}))
#         print('Theta1 ', sess.run(Theta1))
#         print('Bias1 ', sess.run(Bias1))
#         print('Theta2 ', sess.run(Theta2))
#         print('Bias2 ', sess.run(Bias2))
        print('cost ', sess.run(cost, feed_dict={x_: XOR_X, y_: XOR_Y}))

Epoch  0
cost  0.73549473
Epoch  1000
cost  0.69351304
Epoch  2000
cost  0.6934637
Epoch  3000
cost  0.6934328
Epoch  4000
cost  0.6934047
Epoch  5000
cost  0.6933787
Epoch  6000
cost  0.6933545
Epoch  7000
cost  0.69333166
Epoch  8000
cost  0.6933096
Epoch  9000
cost  0.6932881
Epoch  10000
cost  0.69326687
Epoch  11000
cost  0.6932455


KeyboardInterrupt: 